# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
e_layer = nn.Embedding(
    num_embeddings=10,  # vocab size (총 단어 개수/크기)
    embedding_dim=5,    # embedding vector의 차원. (한개 단어를 몇개 값으로 표현.)
    padding_idx=0,      # padding 토큰의 index를 지정.
    #             (pad는 자리만 채우는 토큰이므로 학습이 안되도록 처리하기 위해서.)

    # [PAD](padding) 토큰: 문장들의 토큰 개수를 맞추기 위해서 사용하는 토큰.
    # EX)모든 문장의 토큰수를 10개 로 할 경우. 10개가 안되는 토큰은 나머지를 [PAD] 
    #  토큰으로 채운다.
)

In [5]:
# embedding layer의 weight 조회
e_layer.weight   # word embedding vector들.
e_layer.weight.shape # [10: 단어수, 5:embedding 차원]

torch.Size([10, 5])

In [7]:
# 입력 값 - 정수 tensor(LongTensor-int64)를 입력.
## 한개 문서 : [1, 10, 7, 5] 
                        # 문서를 구성하는 토큰 idx들을 1차원으로 묶어서 전달
input_data = torch.tensor([[1, 3, 2, 7]], dtype=torch.int64)
e = e_layer(input_data)
print(e.shape)
print(e)  # [1: 문서수, 4: 토큰(단어)수, 5:embedding vector 차원]

torch.Size([1, 4, 5])
tensor([[[-0.0095, -0.7010,  0.4702, -0.0711, -0.4668],
         [ 0.5122, -2.9398,  1.3435,  1.8088, -0.0858],
         [ 0.8444, -0.0644,  0.8419, -0.7564, -0.0973],
         [ 1.9628,  0.8114,  0.2339, -1.2946,  1.5357]]],
       grad_fn=<EmbeddingBackward0>)


In [12]:
e_layer.weight[[1, 3, 2, 7]]

tensor([[-0.0095, -0.7010,  0.4702, -0.0711, -0.4668],
        [ 0.5122, -2.9398,  1.3435,  1.8088, -0.0858],
        [ 0.8444, -0.0644,  0.8419, -0.7564, -0.0973],
        [ 1.9628,  0.8114,  0.2339, -1.2946,  1.5357]],
       grad_fn=<IndexBackward0>)

# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

# Dataset, DataLoader 생성

## Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

## 데이터 로딩

In [ ]:
!pip install korpora

In [2]:
import os
import time

from Korpora import Korpora

corpus = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\Playdata\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\

In [3]:
all_input = corpus.get_all_texts()  # inupt: 댓글들 (전체)
all_labels = corpus.get_all_labels()# output: 0:부정, 1긍정(전체)

In [4]:
all_input[:5]

('아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다')

In [5]:
all_labels[:5]

[0, 1, 0, 0, 1]

In [6]:
len(all_input)

200000

In [7]:
corpus.train

NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]

In [8]:
corpus.test

NSMC.test: size=50000
  - NSMC.test.texts : list[str]
  - NSMC.test.labels : list[int]

In [34]:
corpus.test.texts[:10]

('굳 ㅋ',
 'GDNTOPCLASSINTHECLUB',
 '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
 '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
 '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
 '음악이 주가 된, 최고의 음악영화',
 '진정한 쓰레기',
 '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
 '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
 '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..')

In [35]:
corpus.test.labels[:10]

[1, 0, 0, 0, 0, 1, 0, 0, 0, 1]

## 토큰화
1. 형태소 단위 token화(분절)를 먼저 한다.
    - konlpy로 token화 한 뒤 다시 한 문장으로 만든다.
2. 1에서 처리한 corpus를 BPE 로 token화
   
### 전처리 함수

#### 형태소 단위 분절

In [10]:
from konlpy.tag import Okt
import string
import re

okt = Okt()
def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    (stop word 제거)
    """
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", " ", text) #구두점을 공백으로 변환
    tokens = okt.morphs(text, stem=True) # stem:원형복원.
    return ' '.join(tokens)

In [11]:
' '.join(['내가', '어제', '밥을', '먹었다'])
# ' '를 기준으로 리스트의 문자열들을 합친다.

'내가 어제 밥을 먹었다'

In [12]:
# import string
# list(string.punctuation)
f"[{string.punctuation}]"

'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [13]:
text_preprocessing('아 더빙.. 진짜 짜증나네요 목소리')

'아 더빙 진짜 짜증나다 목소리'

In [14]:
### train set/test set 전처리.
train_input = corpus.train.texts

train_texts = [text_preprocessing(txt) for txt in train_input]
train_labels = corpus.train.labels

test_input = corpus.test.texts
test_texts = [text_preprocessing(txt) for txt in test_input]
test_labels = corpus.test.labels

## 전처리된 input을 합치기 (토큰화를 위해서)
all_texts = train_texts + test_texts
print(len(all_texts))

200000


In [15]:
len(train_texts), len(test_texts)

(150000, 50000)

In [16]:
import pickle
import os
os.makedirs('datasets/nsmc', exist_ok=True)
with open('datasets/nsmc/trainset.pkl', 'wb') as fw:
    pickle.dump({"input":train_texts, "output":train_labels}, fw)

with open("datasets/nsmc/testset.pkl", 'wb') as fw2:
    pickle.dump({"input":test_texts, "output":test_labels}, fw2)

### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
    - BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
    - https://huggingface.co/docs/tokenizers/quicktour
    - `pip install tokenizers`

In [17]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30000 # max vocab size
min_frequency = 5  # 5회 이상 나온 단어들만 사전에 추가.

tokenizer = Tokenizer(
    BPE(unk_token='[UNK]')
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency, 
    special_tokens=["[PAD]", "[UNK]"], 
    continuing_subword_prefix="##" # 시작하는 단어가 아닌 경우 ##을 앞에 붙인다.
    # 시작하는 -> 시작,  ##하는
)
# 학습 
# 학습데이터가 메모리에 있을때 train 함수
tokenizer.train_from_iterator(all_texts, trainer=trainer) 

In [18]:
# 총 Vocab size
tokenizer.get_vocab_size()

26738

In [43]:
print(f"[PAD]의 id: {tokenizer.token_to_id('[PAD]')}")
print(tokenizer.id_to_token(300))

[PAD]의 id: 0
北


In [19]:
# 문장 토큰화
print(all_texts[0])
r = tokenizer.encode(all_texts[0])
r.ids

아 더빙 진짜 짜증나다 목소리


[1986, 5881, 5426, 5667, 6087]

In [49]:
r.tokens

['아', '더빙', '진짜', '짜증나다', '목소리']

In [50]:
tokenizer.decode([1986, 5881, 5426, 5667, 6087])

'아 더빙 진짜 짜증나다 목소리'

In [60]:
### Tokenizer를 저장
os.makedirs("saved_model/nsmc", exist_ok=True)
tokenizer.save("saved_model/nsmc/tokenizer.json")

In [20]:
# 불러오기
load_tokenizer = Tokenizer.from_file("saved_model/nsmc/tokenizer.json")
load_tokenizer.encode(all_texts[0]).tokens

['아', '더빙', '진짜', '짜증나다', '목소리']

## Dataset, DataLoader 생성

In [73]:
tokenizer.encode("댓글 목록. 리스트에 ").ids

[6216, 17173, 1, 11373, 3203]

In [23]:
all_texts[:10]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 한 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 그 의 익살스럽다 연기 가 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 가 너무나도 이쁘다 보이다',
 '막 걸음 마 떼다 3 세 부터 초등학교 1 학년 생인 8 살다 영화 ㅋㅋㅋ 별 반개 도 아깝다 움',
 '원작 의 긴장감 을 제대로 살리다 하다',
 '별 반개 도 아깝다 욕 나오다 이응경 길용우 연 기 생활 이 몇 년 인지 정말 발 로 해도 그것 보단 낫다 납치 감금 만 반복 반복 이 드라마 는 가족 도 없다 연기 못 하다 사람 만 모 엿 네',
 '액션 이 없다 재미 있다 몇 안되다 영화',
 '왜케 평점 이 낮다 꽤 볼 만 한 데 헐리우드 식 화려하다 너무 길들이다 있다']

In [27]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    
    def __init__(self, texts, labels, max_length, tokenizer):
        """
        texts: list - 댓글 목록. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
        labels: list - 댓글 감정 목록. 
        max_length: 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.texts = [self.__pad_token_sequences(tokenizer.encode(text).ids) 
                      for text in texts]
        self.labels = labels

    ###########################################################################################
    # id로 구성된 개별 문장 tokenizer list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
    ############################################################################################
    def __pad_token_sequences(self, token_sequences):
        """
        token id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
        max_length 보다 토큰수가 적으면 [PAD] 추가, 많으면 max_length 크기로 줄인다.
            ex) [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
        """
        pad_token = self.tokenizer.token_to_id('[PAD]')
        seq_len = len(token_sequences) # 문장의 토큰개수
        result = None
        if seq_len > self.max_length: # 잘라내기.
            result = token_sequences[:self.max_length]
        else: # [PAD] 토큰을 추가.
            result = token_sequences + ([pad_token] * (self.max_length - seq_len))
        return result
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
        Parameter
            idx: int 조회할 index
        Return
            tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
        """
        txt = self.texts[idx]    # idx번째 댓글 문장 조회
        label = self.labels[idx] # idx번째 정답
        # encode = self.tokenizer.encode(txt) # 토큰화
        # padding_encode = __pad_token_sequences(encode)

        # (input, output) input: Embedding Layer에 입력으로 들어감. -> LongTensor, IntTensor(정수)
        # output: [label]  # loss함수에 입력할 때 (batch, 1)
        return (torch.tensor(txt, dtype=torch.int64),  
                torch.tensor([label], dtype=torch.float32))
        
# BCELoss(): 정답 shape (batch, 1)   - [[1], [0], [0]]
# CrossEntropyLoss(): 정답 shape (batch, ) - [1, 6, 3, ..]       
    

In [70]:
a = [ len(tokenizer.encode(text)) for text in all_texts]  # 모든 문장들의 토큰 개수 리스트.

In [64]:
min(a), max(a)

(0, 89)

In [66]:
import numpy as np
np.quantile(a, q=[0.9, 0.95])

array([29., 41.])

In [26]:
##### Dataset 생성
# max_token수를 30개
MAX_TOKEN = 30
train_set = NSMCDataset(train_texts, train_labels, MAX_TOKEN, tokenizer)
test_set = NSMCDataset(test_texts, test_labels, MAX_TOKEN, tokenizer)
len(train_set), len(test_set)

(150000, 50000)

In [28]:
train_set[10]

(tensor([  540, 11354,   506,  2408,  5414,  5426,  2408,  5414,   119,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([1.]))

In [29]:
### DataLoader
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [30]:
len(train_loader), len(test_loader)

(2343, 782)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [32]:
import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

device = 'cuda' if torch.cuda.is_available() else "cpu"

In [33]:
class NaverCommentClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, 
                 num_layers, bidirectional=True, dropout_rate=0.2):
        
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,   # vocab 크기(총 단어수)
            embedding_dim=embedding_dim, # embedding vector의 차원수(개수)
            padding_idx = 0              # [PAD]의 index. [PAD] 에 대한 weight는 학습하지 않는다.
        )
        # embedding layer의 출력 shape:  (64:batch, seq_len:문장의 토큰수, embedding vector차원수)
        ##  (64, 30, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_size,
            num_layers=num_layers, 
            bidirectional=bidirectional,
            dropout=dropout_rate
        )
        self.dropout = nn.Dropout(dropout_rate)

        # 입력: lstm의 마지막 timestep의 output   out, (h, c) = lstm(X)
        #  i_features = hidden_size * 2 if bidirectional else hidden_size
        if bidirectional == True:
            i_features = hidden_size * 2
        else:
            i_features = hidden_size
        self.classifier = nn.Linear(i_features, 1)  # out_features: 1 - 긍정일 확률
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        # X:  [batch_size, seq_len(문장의 토큰수)]   [64, 30]
        embedding_vector = self.embedding(X)
        # embedding_vector: [batch_size, seq_len, embedding_dim] -> [seq_len, batch_size, embedding_dim]
        embedding_vector = embedding_vector.transpose(1, 0)  # batch축과 seq_len 축을 바꾸기.
        output, _ = self.lstm(embedding_vector)
        # output: [seq_len, batch_size, hidden_size * (2 if bidirectional else 1)] 
#                                                                        => 마지막 output을 추출
        output = output[-1] # shape: (batch_size, hidden_size)
        output = self.dropout(output)
        output = self.classifier(output)
        last_output = self.sigmoid(output)
        return last_output

## 모델 생성

In [34]:
model = NaverCommentClassifier(
    vocab_size=tokenizer.get_vocab_size(), 
    embedding_dim=100, 
    hidden_size=32,
    num_layers=2,
    bidirectional=True,
    dropout_rate=0.3
)

In [85]:
print(model)

NaverCommentClassifier(
  (embedding): Embedding(26739, 100, padding_idx=0)
  (lstm): LSTM(100, 32, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifer): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [91]:
i = torch.randint(1, 10, (10, 30), dtype=torch.int64)
summary(model, input_data=i)

Layer (type:depth-idx)                   Output Shape              Param #
NaverCommentClassifier                   [10, 1]                   --
├─Embedding: 1-1                         [10, 30, 100]             2,673,900
├─LSTM: 1-2                              [30, 10, 64]              59,392
├─Dropout: 1-3                           [10, 64]                  --
├─Linear: 1-4                            [10, 1]                   65
├─Sigmoid: 1-5                           [10, 1]                   --
Total params: 2,733,357
Trainable params: 2,733,357
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 44.56
Input size (MB): 0.00
Forward/backward pass size (MB): 0.39
Params size (MB): 10.93
Estimated Total Size (MB): 11.33

In [35]:
#학습 전에 추론
x, y = next(iter(train_loader))
y_hat = model(x)

In [37]:
y_hat.shape

torch.Size([64, 1])

## 학습

### Train/Test 함수 정의

In [41]:
def train(model, dataloader, loss_fn, optimizer, device='cpu'):
    # 1 epoch 학습
    model = model.to(device)
    model.train()
    loss_list = []
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        # 추론
        pred = model(X)
        # loss 계산
        loss = loss_fn(pred, y)
        # gradient 계산
        loss.backward()
        # 파라미터 업데이트
        optimizer.step()
        # 파라미터 초기화
        optimizer.zero_grad()
        loss_list.append(loss.item())
    return sum(loss_list)/len(dataloader)  # 1 에폭 train loss: step loss들 평균을 반환.

In [46]:
def test(model, dataloader, loss_fn, device="cpu"):
    model = model.to(device)
    model.eval()
    loss_list = []
    acc_list = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred_proba = model(X) #  양성일 확률
            pred_label = (pred_proba > 0.5).type(torch.int32) # bool -> int (False: 0, True: 1)
            loss = loss_fn(pred_proba, y)
            loss_list.append(loss.item())
            acc_list += (y == pred_label).sum().item()
        return sum(loss_list)/len(dataloader), acc_list/len(dataloader.dataset) # 검증 loss, accuracy 

### Train

In [43]:
lr = 0.01
epochs = 1
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [47]:
for epoch in range(epochs):
    # 학습
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = test(model, test_loader, loss_fn, device)
    print(f"[{epoch}/{epochs}] train loss: {train_loss}, val_loss: {val_loss}, val_acc: {val_acc}")

[0/1] train loss: 0.35809781592827394, val_loss: 0.36691004691450185, val_acc: 0.83836


## 모델저장

In [48]:
torch.save(model, 'saved_model/nsmc_model.pt')

# 서비스

## 전처리 함수들

In [51]:
from konlpy.tag import Okt
okt = Okt()
def text_preprocessing(text):
    # 한문장(문서) 전처리.
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]+", ' ', text)
    return ' '.join(okt.morphs(text, stem=True))

In [52]:
def pad_token_sequences(token_sequences, max_length):
    """padding 처리 메소드."""
    pad_token = tokenizer.token_to_id('[PAD]')  
    seq_length = len(token_sequences)           
    result = None
    if seq_length > max_length:                 
        result = token_sequences[:max_length]
    else:                                            
        result = token_sequences + ([pad_token] * (max_length - seq_length))
    return result

In [53]:
def predict_data_preprocessing(text_list):
    """
    모델에 입력할 수있는 input data를 생성
    Parameter:
        text_list: list - 추론할 댓글리스트
    Return
        torch.LongTensor - 댓글 token_id tensor
    """
    # 전처리
    text_list = [text_preprocessing(text) for text in text_list]
    # 토큰화
    tokens = [tokenizer.encode(text).ids for text in text_list]
    # 토큰 + 패딩
    pad_tokens = [pad_token_sequences(token, MAX_TOKEN) for token in tokens]
    return torch.tensor(pad_tokens, dtype=torch.int64)

## 추론

In [54]:
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

torch.Size([9, 30])

In [61]:
# 추론
model.eval()
model.to(device)
with torch.no_grad():
    pred = model(input_tensor)
    for txt, prob in zip(comment_list, pred):
        label = "긍정적 댓글" if prob.item() > 0.5 else "부정적 댓글"
        p = prob.item() if prob.item() > 0.5 else (1-prob).item()
        print(txt, label, p, sep="\t")

아 진짜 재미없다.	부정적 댓글	0.9846872687339783
여기 식당 먹을만 해요	부정적 댓글	0.7542984485626221
이걸 영화라고 만들었냐?	부정적 댓글	0.9461966753005981
기대 안하고 봐서 그런지 괜찮은데.	긍정적 댓글	0.753645122051239
이걸 영화라고 만들었나?	부정적 댓글	0.9461966753005981
아! 뭐야 진짜.	부정적 댓글	0.9483587741851807
재미있는데.	긍정적 댓글	0.950312077999115
연기 짱 좋아. 한번 더 볼 의향도 있다.	긍정적 댓글	0.9704426527023315
뭐 그럭저럭	부정적 댓글	0.9635049104690552


In [63]:
comment_list = [input("댓글:")]
input_tensor = predict_data_preprocessing(comment_list)

model.eval()
model.to(device)
with torch.no_grad():
    pred = model(input_tensor)
    for txt, prob in zip(comment_list, pred):
        label = "긍정적 댓글" if prob.item() > 0.5 else "부정적 댓글"
        p = prob.item() if prob.item() > 0.5 else (1-prob).item()
        print(txt, label, p, sep="\t")

댓글: ㅣㅏㅁ어리만어리;ㅁㄴㅇ러;ㅣㅁㅇ


ㅣㅏㅁ어리만어리;ㅁㄴㅇ러;ㅣㅁㅇ	부정적 댓글	0.610584020614624
